### Download data

In [ ]:
!wget https://lfs.aminer.cn/misc/moocdata/data/mooccube2/relations/user-video.json

--2024-06-04 01:05:11--  https://lfs.aminer.cn/misc/moocdata/data/mooccube2/relations/user-video.json
Resolving lfs.aminer.cn (lfs.aminer.cn)... 159.27.2.14
Connecting to lfs.aminer.cn (lfs.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3179221944 (3.0G) [application/json]
Saving to: ‘user-video.json’

user-video.json     100%[===================>]   2.96G  29.8MB/s    in 1m 48s  

2024-06-04 01:07:00 (28.0 MB/s) - ‘user-video.json’ saved [3179221944/3179221944]



In [ ]:
!wget https://lfs.aminer.cn/misc/moocdata/data/mooccube2/relations/concept-video.txt

--2024-06-04 01:07:00--  https://lfs.aminer.cn/misc/moocdata/data/mooccube2/relations/concept-video.txt
Resolving lfs.aminer.cn (lfs.aminer.cn)... 159.27.2.14
Connecting to lfs.aminer.cn (lfs.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40378364 (39M) [text/plain]
Saving to: ‘concept-video.txt’

concept-video.txt   100%[===================>]  38.51M  21.0MB/s    in 1.8s    

2024-06-04 01:07:02 (21.0 MB/s) - ‘concept-video.txt’ saved [40378364/40378364]



In [ ]:
!wget https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/video.json

--2024-06-04 01:07:02--  https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/video.json
Resolving lfs.aminer.cn (lfs.aminer.cn)... 159.27.2.14
Connecting to lfs.aminer.cn (lfs.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 608069601 (580M) [application/json]
Saving to: ‘video.json’

video.json          100%[===================>] 579.90M  28.4MB/s    in 19s     

2024-06-04 01:07:22 (30.5 MB/s) - ‘video.json’ saved [608069601/608069601]



In [ ]:
!wget https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/course.json

--2024-06-04 01:07:22--  https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/course.json
Resolving lfs.aminer.cn (lfs.aminer.cn)... 159.27.2.14
Connecting to lfs.aminer.cn (lfs.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44660123 (43M) [application/json]
Saving to: ‘course.json’

course.json         100%[===================>]  42.59M  15.3MB/s    in 2.8s    

2024-06-04 01:07:25 (15.3 MB/s) - ‘course.json’ saved [44660123/44660123]



In [ ]:
!wget https://lfs.aminer.cn/misc/moocdata/data/mooccube2/relations/concept-course.txt

--2024-06-04 01:07:25--  https://lfs.aminer.cn/misc/moocdata/data/mooccube2/relations/concept-course.txt
Resolving lfs.aminer.cn (lfs.aminer.cn)... 159.27.2.14
Connecting to lfs.aminer.cn (lfs.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19483276 (19M) [text/plain]
Saving to: ‘concept-course.txt’

concept-course.txt  100%[===================>]  18.58M  16.6MB/s    in 1.1s    

2024-06-04 01:07:27 (16.6 MB/s) - ‘concept-course.txt’ saved [19483276/19483276]



In [ ]:
!wget https://lfs.aminer.cn/misc/moocdata/data/mooccube2/relations/video_id-ccid.txt

--2024-06-04 01:07:27--  https://lfs.aminer.cn/misc/moocdata/data/mooccube2/relations/video_id-ccid.txt
Resolving lfs.aminer.cn (lfs.aminer.cn)... 159.27.2.14
Connecting to lfs.aminer.cn (lfs.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119740314 (114M) [text/plain]
Saving to: ‘video_id-ccid.txt’

video_id-ccid.txt   100%[===================>] 114.19M  28.4MB/s    in 4.6s    

2024-06-04 01:07:32 (24.9 MB/s) - ‘video_id-ccid.txt’ saved [119740314/119740314]



### Main processing

In [ ]:
import dask.dataframe as dd

In [ ]:
import pandas as pd
%load_ext cudf.pandas

In [ ]:
import numpy as np

#### Get video self-feature

In [ ]:
def read_txt_ccid_video_relationship(txt_file):
    content = open(txt_file, 'r')
    data = []
    for line in content.readlines():
        concept, ccid = line.split('\t')
        ccid = ccid.removesuffix('\n')
        data.append([concept, ccid])
    dataframe = pd.DataFrame(np.array(data), columns=['video_id', 'ccid'])
    return dataframe

In [ ]:
ccid_video_relations = read_txt_ccid_video_relationship('video_id-ccid.txt')

In [ ]:
ccid_video_relations

In [ ]:
len(ccid_video_relations)

In [ ]:
videos = pd.read_json('video.json', lines=True)

In [ ]:
videos.columns

In [ ]:
videos.drop(['name', 'text'], axis=1, inplace=True)

In [ ]:
total_duration = videos.apply(lambda series: max(series['end']) - min(series['start']), axis=1)
# total_duration = pd.DataFrame([videos['ccid'], total_duration], columns=['ccid', 'value'])

In [ ]:
videos.insert(len(videos.columns), column='total_duration', value=total_duration)
videos.drop(['start', 'end'], axis=1, inplace=True)

In [ ]:
final_data_video = ccid_video_relations.merge(videos, 'inner', on='ccid')
final_data_video

In [ ]:
final_data_video.drop('ccid', axis=1).to_csv('video_with_duration.csv')

In [ ]:
final_data_video = pd.read_csv("video_with_duration.csv").drop('Unnamed: 0', axis='columns').drop_duplicates()

In [ ]:
final_data_video

In [ ]:
#@title find relations between course and video (course-video)
# courses = pd.read_json('https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/course.json',
#                        lines=True,)
# courses = courses.rename({'id': 'course_id'}, axis='columns')
# course_video = pd.DataFrame([], columns=['course_id', 'resource_id'])
# for i, course in courses.iterrows():
#     resource = np.array(course['resource'])
#     resources = pd.DataFrame([], columns=['titles', 'resource_id',	'chapter'])

#     for row in resource:
#         resources = pd.concat([resources, pd.DataFrame(row)], ignore_index=True)

#     resources = resources[resources.resource_id.str.contains('V')].drop_duplicates(['resource_id'])
#     resources['course_id'] = [course['course_id']] * resources.shape[0]

#     course_video = pd.concat(
#         [course_video, resources[['course_id', 'resource_id']]],
#         ignore_index=True
#     )

# course_video = course_video.drop_duplicates()

!gdown 1q-aorcdG9BI0r3a2WLrPo7NxVlpA95kl
course_video = pd.read_csv('course-video.csv').drop('Unnamed: 0', axis='columns')

In [ ]:
video_linked_in_course = pd.DataFrame(course_video.loc[:,'resource_id'])
video_linked_in_course['total_duration'] = [0.] * video_linked_in_course.shape[0]
video_linked_in_course = video_linked_in_course.rename({'resource_id': 'video_id'}, axis='columns')
video_linked_in_course

In [ ]:
# convert to array for faster performance
a_course_video = np.array(course_video['resource_id'].values)
a_course_video
# only keep video in course
video_in_course = final_data_video[final_data_video['video_id'].isin(a_course_video)]
video_in_course

In [ ]:
final_video_in_course = pd.concat([
        video_in_course,
        video_linked_in_course[~video_linked_in_course['video_id'].isin(video_in_course['video_id'].values)]
    ], axis=0)

In [ ]:
final_video_in_course

In [ ]:
video_in_course.to_csv('video_in_course_w_duration.csv')

In [ ]:
final_video_in_course.to_csv('final_video_in_course_w_duration.csv')

In [ ]:
# # final_data_video.set_index('video_id', inplace=True)
final_video_in_course.set_index('video_id', inplace=True)
# video_linked_in_course.set_index('video_id', inplace=True)

In [ ]:
video_not_in_course = set() # store video ids that not in course
def get_duration(video_id):
    try:
        return final_video_in_course['total_duration'][video_id]
    except:
        # print(f"{video_id} not in video_in_course")
        video_not_in_course.add(video_id)
        return None

#### Extraction function for single line

In [ ]:
def segment_feature_extraction(x: dict):
    segments = x['segment']
    num_of_pauses = len(segments) - 1

    total_duration = get_duration(x['video_id'])
    # không xử lý các video không có lưu trong course, vì sau này thông tin này sẽ không cần thiết
    if total_duration is None:
        # it means that the information of this video is not in the course, we can ignore this one
        return None

    total_watched_duration = 0
    total_weighted_speed = 0
    total_watching_time_with_speed = 0
    total_seeking_time = 0
    num_of_skips = 0

    for i, seg in enumerate(segments):

        total_watched_duration += np.abs(seg['end_point'] - seg['start_point'])
        total_weighted_speed += np.abs(seg['end_point'] - seg['start_point'])
        total_watching_time_with_speed += (seg['end_point'] - seg['start_point']) / seg['speed']

        if i < 1: continue

        if segments[i]['start_point'] > segments[i-1]['end_point']:
            num_of_skips += 1

        if segments[i]['start_point'] < segments[i-1]['end_point']:
            total_seeking_time += segments[i]['start_point'] - segments[i-1]['end_point']


    mean_watching_speed = total_weighted_speed / total_watched_duration if total_watched_duration != 0 else 0
    total_watching_time = total_watched_duration
    real_watching_time = total_watching_time + total_seeking_time

    timestamps = [seg['end_point'] for seg in segments] + [seg['start_point'] for seg in segments] + [total_duration]

    total_duration = max(timestamps)
    if total_duration == 0:
        print(segments)
    completion_rate = real_watching_time / total_duration

    features = {
        'video_id': x['video_id'],
        'num_of_pauses': num_of_pauses,
        'num_of_skips': num_of_skips,
        'mean_watching_speed': mean_watching_speed,
        'total_watching_time': total_watching_time,
        'total_watching_time_with_speed': total_watching_time_with_speed,
        'real_watching_time': real_watching_time,
        'completion_rate': completion_rate
    }
    return pd.Series(features)

In [ ]:
def full_feature_extraction(x: list, user_id: str):
    data = pd.DataFrame(columns=['video_id', 'num_of_pauses', 'num_of_skips',
                                 'mean_watching_speed', 'total_watching_time',
                                 'total_watching_time_with_speed', 'real_watching_time', 'completion_rate'])
    list_data = []
    for vid in x:
        features = segment_feature_extraction(vid)
        if features is None:
            # ignore the videos not in course
            continue
        features = pd.DataFrame([features])
        features.loc[:, 'user_id'] = user_id
        list_data.append(features)

    if len(list_data) > 0:
        data = pd.concat(list_data, axis=0, ignore_index=True)
    return data

#### Test extraction functions

In [ ]:
x = {'video_id': 'V_1395633',
  'segment': [{'start_point': 130.0,
    'end_point': 190.0,
    'speed': 1.0,
    'local_start_time': 1588431144},
   {'start_point': 220.0,
    'end_point': 250.0,
    'speed': 1.0,
    'local_start_time': 1588431234},
   {'start_point': 478.8,
    'end_point': 508.8,
    'speed': 1.0,
    'local_start_time': 1588437359},
   {'start_point': 655.1,
    'end_point': 692.55,
    'speed': 1.25,
    'local_start_time': 1588437514}]}

segment_feature_extraction(x)

V_1395633 not in final_data_video


In [ ]:
full_test = [{'video_id': 'V_1395633',
  'segment': [{'start_point': 130.0,
    'end_point': 190.0,
    'speed': 1.0,
    'local_start_time': 1588431144},
   {'start_point': 220.0,
    'end_point': 250.0,
    'speed': 1.0,
    'local_start_time': 1588431234},
   {'start_point': 478.8,
    'end_point': 508.8,
    'speed': 1.0,
    'local_start_time': 1588437359},
   {'start_point': 655.1,
    'end_point': 692.55,
    'speed': 1.25,
    'local_start_time': 1588437514}]},
 {'video_id': 'V_1395635',
  'segment': [{'start_point': 135.0,
    'end_point': 170.0,
    'speed': 1.0,
    'local_start_time': 1588438045}]},
 {'video_id': 'V_1395636',
  'segment': [{'start_point': 128.3,
    'end_point': 180.8,
    'speed': 1.5,
    'local_start_time': 1588438278}]},
 {'video_id': 'V_1395639',
  'segment': [{'start_point': 100.0,
    'end_point': 106.25,
    'speed': 1.25,
    'local_start_time': 1588438980},
   {'start_point': 180.0,
    'end_point': 186.25,
    'speed': 1.25,
    'local_start_time': 1588439045}]},
 {'video_id': 'V_6210799',
  'segment': [{'start_point': 9.0,
    'end_point': 22.25,
    'speed': 1.25,
    'local_start_time': 1601012656},
   {'start_point': 28.0,
    'end_point': 271.5,
    'speed': 1.25,
    'local_start_time': 1601012671},
   {'start_point': 279.0,
    'end_point': 331.5,
    'speed': 1.5,
    'local_start_time': 1601012841},
   {'start_point': 346.0,
    'end_point': 356.0,
    'speed': 1.5,
    'local_start_time': 1601012886}]},
 {'video_id': 'V_6210800',
  'segment': [{'start_point': 6.0,
    'end_point': 13.5,
    'speed': 1.5,
    'local_start_time': 1601012899}]}]

full_feature_extraction(full_test)

V_1395633 not in video_in_course
V_1395635 not in video_in_course
V_1395636 not in video_in_course
V_1395639 not in video_in_course
V_6210799 not in video_in_course
V_6210800 not in video_in_course


,video_id,num_of_pauses,num_of_skips,mean_watching_speed,total_watching_time,total_watching_time_with_speed,real_watching_time,completion_rate


#### Extraction functions for test dataset

In [ ]:
# def extracting_chunk(chunk: pd.DataFrame):
#     extraction = chunk.apply(lambda x: full_feature_extraction(x['seq']), axis=1)

#     result = pd.DataFrame(columns=['video_id', 'num_of_pauses', 'num_of_skips',
#                                  'mean_watching_speed', 'total_watching_time',
#                                  'total_watching_time_with_speed', 'real_watching_time', 'completion_rate'])

#     for index, extracts in extraction:
#         result = pd.concat([result, extracts], axis=0, ignore_index=True)

#     return result

In [ ]:
# Bỏ nrows để chạy full dataset
chunk = pd.read_json(
    'user-video.json',
    lines=True,
    # nrows=5000
)

In [ ]:
chunk

,seq,user_id
0,"[{'video_id': 'V_1395633', 'segment': [{'start...",U_112
1,"[{'video_id': 'V_6224262', 'segment': [{'start...",U_150
2,"[{'video_id': 'V_6432959', 'segment': [{'start...",U_172
3,"[{'video_id': 'V_6334508', 'segment': [{'start...",U_189
4,"[{'video_id': 'V_1358540', 'segment': [{'start...",U_197
...,...,...
310355,"[{'video_id': 'V_8156324', 'segment': [{'start...",U_36053916
310356,"[{'video_id': 'V_6268956', 'segment': [{'start...",U_36053934
310357,"[{'video_id': 'V_6388141', 'segment': [{'start...",U_36053946
310358,"[{'video_id': 'V_8244595', 'segment': [{'start...",U_36053952


In [ ]:
!gdown 1BcCMZ7P7AzkWJHw9HvuGu5qgXduavFvI

In [ ]:
user_ids = pd.read_csv('user_ids.csv')
a_user_ids = user_ids.values.reshape((-1))
# only extract rows in user table to optimize performance
user_video = chunk[chunk['user_id'].isin(a_user_ids)]
del chunk
user_video

In [ ]:
# feature_extraction = extracting_chunk(chunk)
extraction = user_video.apply(lambda x: full_feature_extraction(x['seq'], x['user_id']), axis=1)
feature_extraction = pd.concat(np.array(extraction), ignore_index=True)

[{'start_point': 0.0, 'end_point': 0.0, 'speed': 1.0, 'local_start_time': 1603793740}]


<ipython-input-48-35e061f1ba68>:41: RuntimeWarning: invalid value encountered in scalar divide
  completion_rate = real_watching_time / total_duration


[{'start_point': 0.0, 'end_point': 0.0, 'speed': 1.0, 'local_start_time': 1602588310}]
[{'start_point': 0.0, 'end_point': 0.0, 'speed': 1.0, 'local_start_time': 1603793392}]


<ipython-input-48-35e061f1ba68>:41: RuntimeWarning: invalid value encountered in scalar divide
  completion_rate = real_watching_time / total_duration
<ipython-input-48-35e061f1ba68>:41: RuntimeWarning: invalid value encountered in scalar divide
  completion_rate = real_watching_time / total_duration


[{'start_point': 0.0, 'end_point': 0.0, 'speed': 1.0, 'local_start_time': 1602588278}]


...

<ipython-input-48-35e061f1ba68>:41: RuntimeWarning: invalid value encountered in scalar divide
  completion_rate = real_watching_time / total_duration
<ipython-input-48-35e061f1ba68>:41: RuntimeWarning: invalid value encountered in scalar divide
  completion_rate = real_watching_time / total_duration


[{'start_point': 0.0, 'end_point': 0.0, 'speed': 1.0, 'local_start_time': 1601690288}]
[{'start_point': 0.0, 'end_point': 0.0, 'speed': 1.0, 'local_start_time': 1601691205}]


<ipython-input-48-35e061f1ba68>:41: RuntimeWarning: invalid value encountered in scalar divide
  completion_rate = real_watching_time / total_duration
<ipython-input-48-35e061f1ba68>:41: RuntimeWarning: invalid value encountered in scalar divide
  completion_rate = real_watching_time / total_duration


[{'start_point': 0.0, 'end_point': 0.0, 'speed': 1.0, 'local_start_time': 1601693807}]


<ipython-input-48-35e061f1ba68>:41: RuntimeWarning: invalid value encountered in scalar divide
  completion_rate = real_watching_time / total_duration


In [ ]:
feature_extraction

,video_id,num_of_pauses,num_of_skips,mean_watching_speed,total_watching_time,total_watching_time_with_speed,real_watching_time,completion_rate,user_id
0,V_6432959,0,0,1.0,4.7,4.7,4.7,0.001031,U_172
1,V_6392554,0,0,1.0,24.8,24.8,24.8,0.190184,U_1368
2,V_7406237,1,1,1.0,95.0,95.0,95.0,0.263889,U_1819
3,V_7406238,0,0,1.0,40.1,40.1,40.1,0.891111,U_1819
4,V_7406239,0,0,1.0,40.0,40.0,40.0,0.890869,U_1819
...,...,...,...,...,...,...,...,...,...
229044,V_2842363,1,0,1.0,166.0,166.0,148.6,0.131260,U_34712061
229045,V_4536364,2,2,1.0,51.2,51.2,51.2,0.020008,U_34712061
229046,V_655815,0,0,1.0,10.0,5.0,10.0,0.022371,U_34712103
229047,V_655816,0,0,1.0,323.0,161.5,323.0,0.981763,U_34712103


In [ ]:
feature_extraction.sort_values('completion_rate')

,video_id,num_of_pauses,num_of_skips,mean_watching_speed,total_watching_time,total_watching_time_with_speed,real_watching_time,completion_rate,user_id
67381,V_8151926,253,188,1.0,2980.0,-361.0,-12659.0,-18.175162,U_11636590
67063,V_8152139,196,158,1.0,1335.0,753.0,-12638.0,-16.705882,U_11517631
67382,V_8151929,184,134,1.0,1985.5,-158.5,-8256.0,-15.831256,U_11636590
50547,V_7250365,234,189,1.0,1723.5,770.5,-13532.0,-15.438677,U_8242771
50568,V_7250413,232,181,1.0,2505.0,-45.0,-12965.0,-14.791786,U_8242771
...,...,...,...,...,...,...,...,...,...
219600,V_7035459,0,0,0.0,0.0,0.0,0.0,NaN,U_34171704
219605,V_7035471,0,0,0.0,0.0,0.0,0.0,NaN,U_34171704
224895,V_7035459,0,0,0.0,0.0,0.0,0.0,NaN,U_34195651
224898,V_7035466,0,0,0.0,0.0,0.0,0.0,NaN,U_34195651


In [ ]:
final_video_in_course.loc[feature_extraction[feature_extraction['completion_rate'].isna()]['video_id'].values,:]

,total_duration
video_id,
V_7796661,0.0
V_6332694,0.0
V_8016665,0.0
V_6332694,0.0
V_7926476,0.0
V_8156318,0.0
V_7033658,0.0
V_6411448,0.0
V_6425884,0.0


In [ ]:
feature_extraction[feature_extraction['completion_rate'].isna()]['video_id'].values

,video_id,num_of_pauses,num_of_skips,mean_watching_speed,total_watching_time,total_watching_time_with_speed,real_watching_time,completion_rate,user_id
34951,V_7796661,0,0,0.0,0.0,0.0,0.0,NaN,U_4817629
37687,V_6332694,0,0,0.0,0.0,0.0,0.0,NaN,U_5355583
37804,V_8016665,0,0,0.0,0.0,0.0,0.0,NaN,U_5417504
45354,V_6332694,0,0,0.0,0.0,0.0,0.0,NaN,U_7276268
55598,V_7926476,0,0,0.0,0.0,0.0,0.0,NaN,U_8919835
69360,V_8156318,0,0,0.0,0.0,0.0,0.0,NaN,U_11991072
70935,V_7033658,0,0,0.0,0.0,0.0,0.0,NaN,U_12360180
73003,V_6411448,0,0,0.0,0.0,0.0,0.0,NaN,U_12699387
78258,V_6425884,0,0,0.0,0.0,0.0,0.0,NaN,U_13570971
78261,V_6425939,0,0,0.0,0.0,0.0,0.0,NaN,U_13570971


In [ ]:
feature_extraction[feature_extraction['completion_rate'] > 1.0][['video_id', 'user_id']]\
    .groupby('user_id').count()

KeyError: "['user_id'] not in index"

In [ ]:
feature_extraction[feature_extraction['completion_rate'] > 1.0]\
    .sort_values(['completion_rate'], ascending=False)

,video_id,num_of_pauses,num_of_skips,mean_watching_speed,total_watching_time,total_watching_time_with_speed,real_watching_time,completion_rate,user_id
58492,V_4885147,52,52,1.0,5.931199e+07,-5.931199e+07,5.931199e+07,81327.283285,U_9618295
58493,V_4885154,6,6,1.0,7.831600e+06,-7.831600e+06,7.831600e+06,11968.335778,U_9618295
77316,V_7573083,439,123,1.0,5.497460e+04,-5.094280e+04,5.388540e+04,68.863131,U_13416365
193003,V_6777526,93,93,1.0,6.211170e+04,-6.211170e+04,6.211170e+04,49.291088,U_33016719
78651,V_6136317,45,45,1.0,2.781010e+04,-2.781010e+04,2.781010e+04,22.869621,U_13691319
...,...,...,...,...,...,...,...,...,...
38735,V_5936012,4,4,1.0,8.207000e+02,-3.717700e+02,8.207000e+02,1.002915,U_5794775
114077,V_6392105,3,3,1.0,6.803030e+02,-6.595370e+02,6.803030e+02,1.002830,U_27378434
61009,V_7035537,1,1,1.0,7.306000e+02,-1.756000e+02,7.306000e+02,1.002057,U_10340300
13915,V_6299635,2,2,1.0,5.784000e+02,-3.684000e+02,5.784000e+02,1.000976,U_1980873


In [ ]:
feature_extraction[feature_extraction['completion_rate'] < 0.0]\
    .sort_values(['completion_rate'])

,video_id,num_of_pauses,num_of_skips,mean_watching_speed,total_watching_time,total_watching_time_with_speed,real_watching_time,completion_rate,user_id
67381,V_8151926,253,188,1.0,2980.000,-361.000000,-12659.000,-18.175162,U_11636590
67063,V_8152139,196,158,1.0,1335.000,753.000000,-12638.000,-16.705882,U_11517631
67382,V_8151929,184,134,1.0,1985.500,-158.500000,-8256.000,-15.831256,U_11636590
50547,V_7250365,234,189,1.0,1723.500,770.500000,-13532.000,-15.438677,U_8242771
50568,V_7250413,232,181,1.0,2505.000,-45.000000,-12965.000,-14.791786,U_8242771
...,...,...,...,...,...,...,...,...,...
142567,V_5934166,2,1,1.0,161.476,161.476000,-0.215,-0.000464,U_30996172
146472,V_6136362,24,20,1.0,125.000,125.000000,-0.400,-0.000455,U_31040521
70967,V_507613,1,0,1.0,14.252,14.252000,-0.118,-0.000240,U_12363578
112409,V_198561,3,1,1.0,529.940,529.940000,-0.050,-0.000173,U_26614868


In [ ]:
feature_extraction[
    (feature_extraction['completion_rate'] > 1.0) |
     (feature_extraction['completion_rate'] < 0.0)
][['video_id', 'user_id']]\
    .groupby('user_id').count().sort_values(['video_id'], ascending=False).head(20)

,video_id
user_id,
U_31040521,8
U_30943540,7
U_30565876,6
U_16147480,5
U_17703870,5
U_11080917,4
U_18239122,4
U_31771313,4
U_1071338,4


In [ ]:
feature_extraction[
    (feature_extraction['completion_rate'] > 1.0) |
     (feature_extraction['completion_rate'] < 0.0)
]

In [ ]:
final_video_in_course.loc['V_4885147',:]

total_duration    706.68
Name: V_4885147, dtype: float64

In [ ]:
feature_extraction.loc[feature_extraction['user_id']=='U_9618295',:]

,video_id,num_of_pauses,num_of_skips,mean_watching_speed,total_watching_time,total_watching_time_with_speed,real_watching_time,completion_rate,user_id
58492,V_4885147,52,52,1.0,59311987.7,-59311987.7,59311987.7,81327.283285,U_9618295
58493,V_4885154,6,6,1.0,7831600.2,-7831600.2,7831600.2,11968.335778,U_9618295


In [ ]:
feature_extraction.loc[feature_extraction['user_id']=='U_9618295',:]

,video_id,num_of_pauses,num_of_skips,mean_watching_speed,total_watching_time,total_watching_time_with_speed,real_watching_time,completion_rate,user_id
10750,V_4885147,52,52,1.0,59311987.7,-59311987.7,59311987.7,81327.283285,U_9618295
10751,V_4885154,6,6,1.0,7831600.2,-7831600.2,7831600.2,11968.335778,U_9618295


In [ ]:
feature_extraction.loc[feature_extraction['video_id']=='V_4885147',:]

,video_id,num_of_pauses,num_of_skips,mean_watching_speed,total_watching_time,total_watching_time_with_speed,real_watching_time,completion_rate,user_id
10750,V_4885147,52,52,1.0,59311987.70,-5.931199e+07,59311987.70,81327.283285,U_9618295
30059,V_4885147,6,4,1.0,1245.75,8.977350e+02,484.95,0.663724,U_21383576
53371,V_4885147,0,0,1.0,229.90,1.149500e+02,229.90,0.315710,U_31737536


In [ ]:
feature_extraction.to_csv('extract_results_w_user_id.csv')

In [ ]:
import pickle
with open('video_not_in_course.pkl', 'wb') as file:
    pickle.dump(video_not_in_course, file)

In [ ]:
# feature_extraction.drop(
    # feature_extraction[(feature_extraction['completion_rate'] > 1.0) | (feature_extraction['completion_rate'] < 0.0)].index,
    # axis=0,
    # inplace=True
# )

In [ ]:
feature_extraction.groupby('video_id').mean()\
    .to_csv('features_from_video_example.csv')

TypeError: Could not convert U_30786396 to numeric

In [ ]:
data = pd.read_csv('features_from_video_example.csv')

In [ ]:
data

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.metrics import silhouette_score

In [ ]:
models = [KMeans(n_clusters=5), AgglomerativeClustering(n_clusters=5)]
y = [model.fit_predict(data.drop('video_id', axis=1)) for model in models]

In [ ]:
best = np.argmax([silhouette_score(data.drop('video_id', axis=1), label) for label in y])

In [ ]:
y[best]

array([1, 1, 1, ..., 1, 3, 1], dtype=int32)

In [ ]:
# Lưu models[best] về

In [ ]:
data.insert(len(data.columns), column='label', value=y[best])
data

,video_id,num_of_pauses,num_of_skips,mean_watching_speed,total_watching_time,total_watching_time_with_speed,real_watching_time,completion_rate,label
0,V_1000454,0.0,0.0,1.0,75.0,75.0,75.0,0.046606,1
1,V_1026126,1.0,1.0,1.0,120.0,120.0,120.0,0.917431,1
2,V_1026129,0.0,0.0,1.0,68.9,68.9,68.9,0.084592,1
3,V_1026139,3.0,2.0,1.0,679.9,679.9,627.6,0.795034,0
4,V_1026199,0.0,0.0,1.0,65.0,65.0,65.0,0.873656,1
...,...,...,...,...,...,...,...,...,...
12662,V_89542,1.0,1.0,1.0,399.0,399.0,399.0,0.950000,2
12663,V_89543,0.0,0.0,1.0,333.0,333.0,333.0,0.988131,2
12664,V_89544,1.0,1.0,1.0,135.0,135.0,135.0,0.546311,1
12665,V_98547,1.0,1.0,1.5,225.0,150.0,225.0,0.912779,3


In [ ]:
score = data.groupby('label')['completion_rate'].mean()

In [ ]:
score

label
0    0.801614
1    0.315050
2    0.744091
3    0.607655
4    0.779044
Name: completion_rate, dtype: float64

In [ ]:
new_label = np.argsort(score)

In [ ]:
new_label

label
0    1
1    3
2    2
3    4
4    0
Name: completion_rate, dtype: int64

In [ ]:
data['label'] = data['label'].map(new_label)

In [ ]:
data

,video_id,num_of_pauses,num_of_skips,mean_watching_speed,total_watching_time,total_watching_time_with_speed,real_watching_time,completion_rate,label
0,V_1000454,0.0,0.0,1.0,75.0,75.0,75.0,0.046606,3
1,V_1026126,1.0,1.0,1.0,120.0,120.0,120.0,0.917431,3
2,V_1026129,0.0,0.0,1.0,68.9,68.9,68.9,0.084592,3
3,V_1026139,3.0,2.0,1.0,679.9,679.9,627.6,0.795034,1
4,V_1026199,0.0,0.0,1.0,65.0,65.0,65.0,0.873656,3
...,...,...,...,...,...,...,...,...,...
12662,V_89542,1.0,1.0,1.0,399.0,399.0,399.0,0.950000,2
12663,V_89543,0.0,0.0,1.0,333.0,333.0,333.0,0.988131,2
12664,V_89544,1.0,1.0,1.0,135.0,135.0,135.0,0.546311,3
12665,V_98547,1.0,1.0,1.5,225.0,150.0,225.0,0.912779,4


### TO-DO:

- Trích xuất course-video. (done)
- Tính trung bình điểm số các video trong course (`np.mean(data['label'])`), gọi là `course['score']`. Các video không có điểm số được tính là `0` điểm.
- Gắn nhãn từ tốt nhất đến tệ nhất (4 -> 0) với tỉ lệ: 10%, 20%, 40%, 20%, 10% dựa trên `'score'`.
- Lưu label vào `'video_label.json'`.

# CHANGE LOG

- Chỉ tính toán các video có video_id được gắn với course, và các segments của users có trong bảng user. Lý do: sau này khi tính cho course thì nếu video_id không gắn với course thì việc tính toán là vô nghĩa, và nếu sử dụng hoạt động của user không có trong bảng user thì sau này tính toán gì khác cũng sẽ bị không nhất quán

- Cải thiện performance của `segment_feature_extraction`: nhiều vòng for tương tự nhau => gộp lại tính chung trong 1 vòng for

- Cải thiện `extracting_chunk`: cùng concat và tính toán trong cùng 1 vòng for, đem ra ngoài hàm để dễ monitor tiến độ tính toán